## DEMO VER 1.1

시간별로 잘린 문장을 -> T5 모델로 요약을 한후 -> 중요 문장과 주요 단어를 추출하여 노션으로 정리한다.


In [ ]:
from google.colab import drive
drive.mount("/content/drive")

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


### Segmentation by time

In [ ]:
import pandas as pd
import re
import datetime

d = {
    'timestamp': [],
    'user': [],
    'chat': [],
}
def is_unnecessary_chat(line: str) -> bool:
    """실제 채팅 내용이 아닌 것들 확인"""
    if "님이 나갔습니다." in line:
        return True
    elif "님이 들어왔습니다." in line:
        return True
    elif "채팅방 관리자가 메시지를 가렸습니다." in line:
        return True
    elif "사진" == line:
        return True
    elif "이모티콘" in line:
        return True
    elif "삭제된 메시지입니다." in line:
        return True
    elif "부탁드립니" in line:
        return True
    elif "안녕하세" in line:
        return True
    elif "선착순 선물에 당첨되었어요" in line:
        return True
    else:
        return False

In [ ]:
with open("/content/drive/MyDrive/23-1 데캡디/데이터/데이터.txt", "r", encoding="utf-8") as f:
    content = f.read()
    lines = content.split("2023년 ")  # 한 대화에 \n이 여러번 들어간 것들이 있어서 일단 요걸로 split
    # 텍스트 파일의 맨 윗부분, 대화 내용에 2023년이 들어간 경우 삭제함.
    #pattern = r"^(오전|오후) \d{1,2}:\d{2}$"

    for line in lines[1:]:
        if line == '':
            continue
        elif is_unnecessary_chat(line):
            continue

        try:
            timestamp, user_chat = line.split(", ", 1)
            user, chat = user_chat.split(" : ", 1)
        except ValueError as e:
            # print(e, line)
            continue

        timestamp = "2023년 " + timestamp
        d['timestamp'].append(timestamp)
        d['user'].append(user)
        d['chat'].append(chat[:-1])


df = pd.DataFrame(data=d)

df['timestamp'] = df['timestamp'].str.replace('오전', 'AM')
df['timestamp'] = df['timestamp'].str.replace('오후', 'PM')

df["timestamp"] = pd.to_datetime(df["timestamp"], format="%Y년 %m월 %d일 %p %I:%M")
df['prev_timestamp'] = df['timestamp'].shift(1)
df['timedelta'] = df['timestamp']-df['prev_timestamp']

thld_5min = []
thld_10min = []
thld_15min = []
thld_20min = []

# 클러스터링 기준 번호. 0부터 시작
thld_5min_cnt = 0
thld_10min_cnt = 0
thld_15min_cnt = 0
thld_20min_cnt = 0

for i, row in df[['timedelta']].iterrows():
    if row['timedelta'] > datetime.timedelta(minutes=5):
        thld_5min_cnt += 1
    if row['timedelta'] > datetime.timedelta(minutes=10):
        thld_10min_cnt += 1
    if row['timedelta'] > datetime.timedelta(minutes=15):
        thld_15min_cnt += 1
    if row['timedelta'] > datetime.timedelta(minutes=20):
        thld_20min_cnt += 1

    thld_5min.append(thld_5min_cnt)
    thld_10min.append(thld_10min_cnt)
    thld_15min.append(thld_15min_cnt)
    thld_20min.append(thld_20min_cnt)

df['thld_5min'] = thld_5min
df['thld_10min'] = thld_10min
df['thld_15min'] = thld_15min
df['thld_20min'] = thld_20min

print(df)
# df.to_excel("/content/drive/MyDrive/23-1 데캡디/데이터result.xlsx")

                timestamp                  user  \
0     2023-01-04 20:35:00   mintbear |미드노션워플|기획   
1     2023-01-04 20:36:00            제이/기획/모빌리티   
2     2023-01-04 20:37:00        김태현 | PM | 글쓰기   
3     2023-01-04 20:38:00        김태현 | PM | 글쓰기   
4     2023-01-04 20:47:00  이태극 | 크리에이터 | 유튭,블로그   
...                   ...                   ...   
18273 2023-03-17 11:22:00               유민수|자영업   
18274 2023-03-17 11:59:00    루니 | 마케팅 | 노코드&글쓰기   
18275 2023-03-17 12:37:00        퍼기/ai 전반/ 엔지니어   
18276 2023-03-17 12:38:00    송준용 | 디지털 마케팅 | 대표   
18277 2023-03-17 12:39:00            박형민/지앤텍/서울   

                                                    chat      prev_timestamp  \
0                                                  맞습니다~                 NaT   
1                                    저작권은 그럼 AI한테 있는걸까요? 2023-01-04 20:35:00   
2      아직 저작권은 정해진 바가 없습니다. 현재 OpenAI와 microsoft가 미국에... 2023-01-04 20:36:00   
3      이게 판결이 나는데도 3-4년 걸릴텐데 그 정도면 …. 소송 전 대비해서 엄청나게 ... 2023-01-04

In [ ]:
df.columns

Index(['timestamp', 'user', 'chat', 'prev_timestamp', 'timedelta', 'thld_5min',
       'thld_10min', 'thld_15min', 'thld_20min'],
      dtype='object')

## 텍스트 EDA 및 전처리

### 데이터 전처리 진행_반갑습니다. 감사합니다 등을 기준으로 대화 나누기.

In [ ]:
df_grouped_5 = df.groupby('thld_5min').agg({'chat': ' [sep] '.join})
df_5 = df_grouped_5
df_grouped_10 = df.groupby('thld_10min').agg({'chat': ' [sep] '.join})
df_10 = df_grouped_10
df_grouped_15 = df.groupby('thld_15min').agg({'chat': ' [sep] '.join})
df_15 = df_grouped_15
df_grouped_20 = df.groupby('thld_20min').agg({'chat': ' [sep] '.join})
df_20 = df_grouped_20

In [ ]:
import pandas as pd

new_rows = []
for index, row in df_5.iterrows():
    chat_text = row["chat"]
    split_texts = chat_text.split("반갑습니다")
    new_rows.append({"chat": split_texts[0].strip()})
    if split_texts[1:]:
        new_rows.append({"chat": "반갑습니다"})
    for text in split_texts[1:]:
        new_rows.append({"chat": text.strip()})

new_df_5 = pd.DataFrame(new_rows)
df_5 = new_df_5

new_rows = []
for index, row in df_5.iterrows():
    chat_text = row["chat"]
    split_texts = chat_text.split("감사합니다")
    new_rows.append({"chat": split_texts[0].strip()})
    if split_texts[1:]:
        new_rows.append({"chat": "감사합니다"})
    for text in split_texts[1:]:
        new_rows.append({"chat": text.strip()})

new_df_5 = pd.DataFrame(new_rows)

In [ ]:
print(new_df_5.head(20))

                                                 chat
0   맞습니다~ [sep] 저작권은 그럼 AI한테 있는걸까요? [sep] 아직 저작권은 ...
1   사진 [sep] 오늘 웨비나에서 배우고 기억에 남은 것들 간략하게 정리해봤어요.\n...
2                                               감사합니다
3   😄 [sep] 강연 듣고, 나중에 정리해야지- 하면 안되더라구요. 너무 멋집니다 [...
4   ! [sep] 이런것도 알려주셨나요? 이동하면서 들어서 중간중건에 못들어서요 [se...
5   오늘 참여 못해서 너무 아쉬운데 혹시 다음 강의 계획도 있으실지요~?  [sep] ...
6                                               감사합니다
7                                                 ~^^
8   어설픈 편법으로 (Airtable 샘플에 부족하긴 하지만), Midjourney-D...
9                                               감사합니다
10  !! [sep] 작은 도움이라도 됐다면 제가 외려 감사하죠 [sep] ㅎㅎㅎ [se...
11  ! IFTTT는 조금 기능이 한정적인 느낌이라 쓰다가 말았었는데 Zapier도 한번...
12  태현님 어제 세브방에 모집하신 스터디랑 다른 것인가요? [sep] 넵 지금 그 스터...
13                                                   
14                                              반갑습니다
15                                              사슴님 ~
16  정말 감사하게도 오늘 세미나를 해 주신 준용님께서 부방장을 맡아 주셨습니다 ~ 같이...
17                          

In [ ]:
# Reset the index of the DataFrame
new_df_5.reset_index(drop=True, inplace=True)

# Create a boolean mask for rows where the length of 'chat' is less than 20
mask = new_df_5['chat'].str.len() < 50

# Apply the mask to filter the DataFrame and keep rows where the condition is True
filtered_df = new_df_5[~mask]

# Print the filtered DataFrame
print(filtered_df)
df_5 = filtered_df

                                                   chat
0     맞습니다~ [sep] 저작권은 그럼 AI한테 있는걸까요? [sep] 아직 저작권은 ...
1     사진 [sep] 오늘 웨비나에서 배우고 기억에 남은 것들 간략하게 정리해봤어요.\n...
3     😄 [sep] 강연 듣고, 나중에 정리해야지- 하면 안되더라구요. 너무 멋집니다 [...
4     ! [sep] 이런것도 알려주셨나요? 이동하면서 들어서 중간중건에 못들어서요 [se...
5     오늘 참여 못해서 너무 아쉬운데 혹시 다음 강의 계획도 있으실지요~?  [sep] ...
...                                                 ...
3996  사진 [sep] 버블 언제 꺼지나 했는데 직접 나서서 [sep] 사진 [sep] 타...
4013  [sep] 세상에 추파춥스 100명이라니 ㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋ [sep] 수많은 회원...
4014  !! 🏅 [sep] @장병준 | 사업개발 | 노코드 장피엠님이 출판사 추천해 주셨어...
4016  ! [sep] 사진 [sep] 헛 벌써 받으셨어요? [sep] 사전예약햇죠 [sep...
4017  ! [sep] 대단하고 축하드립니다 ! : ) [sep] 전 예스24^^ [sep]...

[2247 rows x 1 columns]


### Summarization by T5

In [ ]:
!pip install -q wordcloud
!pip install transformers

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM
import nltk
nltk.download('punkt')

model_dir = "lcw99/t5-base-korean-text-summary"
tokenizer = AutoTokenizer.from_pretrained(model_dir)
model = AutoModelForSeq2SeqLM.from_pretrained(model_dir)

max_input_length = 512

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [ ]:
import torch
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model.to('cuda')

T5ForConditionalGeneration(
  (shared): Embedding(50358, 768)
  (encoder): T5Stack(
    (embed_tokens): Embedding(50358, 768)
    (block): ModuleList(
      (0): T5Block(
        (layer): ModuleList(
          (0): T5LayerSelfAttention(
            (SelfAttention): T5Attention(
              (q): Linear(in_features=768, out_features=768, bias=False)
              (k): Linear(in_features=768, out_features=768, bias=False)
              (v): Linear(in_features=768, out_features=768, bias=False)
              (o): Linear(in_features=768, out_features=768, bias=False)
              (relative_attention_bias): Embedding(32, 12)
            )
            (layer_norm): T5LayerNorm()
            (dropout): Dropout(p=0.1, inplace=False)
          )
          (1): T5LayerFF(
            (DenseReluDense): T5DenseGatedActDense(
              (wi_0): Linear(in_features=768, out_features=2048, bias=False)
              (wi_1): Linear(in_features=768, out_features=2048, bias=False)
              (wo):

In [ ]:
summary_data = pd.DataFrame(columns=['text', 'summary', 'url', 'url_title', 'url_desc'])
#df to df_5, df_10, df_15, df_20
summary_data['text'] = df_5['chat']

In [ ]:
import re
import requests
from bs4 import BeautifulSoup

# define regex pattern for URLs
url_pattern = re.compile(r'http[s]?://(?:[a-zA-Z]|[0-9]|[$-_@.&+]|[!*\(\),]|(?:%[0-9a-fA-F][0-9a-fA-F]))+')

count = 0
# loop through rows of dataframe
for index, row in summary_data.iterrows():
    # find URLs in 'text' column
    urls = re.findall(url_pattern, row['text'])

    if urls:
      summary_data.at[index, 'url'] = urls
      urls_info_list = []
      headers = {'User-Agent' : 'Mozilla/5.0 (Windows NT 10.0; Win64; x64)AppleWebKit/537.36 (KHTML, like Gecko) Chrome/73.0.3683.86 Safari/537.36'}

      for url in urls:
        try:
          print("url")
          print(url)
          data = requests.get(url,headers=headers)
          soup = BeautifulSoup(data.text, 'html.parser')

          try:
            title = soup.select_one('meta[property="og:title"]')['content']
          except:
            title = 'No Title'

          try:
            desc = soup.select_one('meta[property="og:description"]')['content']
          except:
            desc = 'No desc'

          urls_info = {}
          urls_info['title'] = title
          urls_info['desc'] = desc
          urls_info_list.append(urls_info)
          print(str(urls_info_list))
        except:
          summary_data.at[index, 'url_info'] = "BLANK"

        summary_data.at[index, 'url_title'] = title
        summary_data.at[index, 'url_desc'] = desc
    else:
        summary_data.at[index, 'url'] = "BLANK"
        summary_data.at[index, 'url_title'] = "BLANK"
        summary_data.at[index, 'url_desc'] = "BLANK"

    # if urls:
    #     summary_data.at[index, 'url'] = urls[0]
    # else:
    #     summary_data.at[index, 'url'] = "BLANK"
    count +=1
    if count > 100:
       break
summary_data.head(25)

url
https://www.youtube.com/watch?v=on623d5EfPw
[{'title': 'Watch ChatGPT Build a Finance Startup', 'desc': "Let's build a demo Finance application using ChatGPT! In this video, we'll use ChatGPT to build a cross platform mobile app that connects to a users bank tra..."}]
url
https://lexica.art/
[{'title': 'Lexica', 'desc': 'The Stable Diffusion search engine'}]
url
https://blog.naver.com/pnumber/222975208525
[{'title': 'No Title', 'desc': 'No desc'}]
url
https://twitter.com/mathemagic1an/status/1610147883894079488
[{'title': 'No Title', 'desc': 'No desc'}]
url
https://twitter.com/RReverser/status/1599184983402446848
[{'title': 'No Title', 'desc': 'No desc'}, {'title': 'No Title', 'desc': 'No desc'}]
url
https://twitter.com/goodside/status/1561549768987496449
[{'title': 'No Title', 'desc': 'No desc'}, {'title': 'No Title', 'desc': 'No desc'}, {'title': 'No Title', 'desc': 'No desc'}]
url
https://twitter.com/goodside/status/1610552172038737920/photo/1
[{'title': 'No Title', 'desc': 'No 

,text,summary,url,url_title,url_desc
0,맞습니다~ [sep] 저작권은 그럼 AI한테 있는걸까요? [sep] 아직 저작권은 ...,NaN,BLANK,BLANK,BLANK
1,사진 [sep] 오늘 웨비나에서 배우고 기억에 남은 것들 간략하게 정리해봤어요.\n...,NaN,BLANK,BLANK,BLANK
3,"😄 [sep] 강연 듣고, 나중에 정리해야지- 하면 안되더라구요. 너무 멋집니다 [...",NaN,BLANK,BLANK,BLANK
4,! [sep] 이런것도 알려주셨나요? 이동하면서 들어서 중간중건에 못들어서요 [se...,NaN,BLANK,BLANK,BLANK
5,오늘 참여 못해서 너무 아쉬운데 혹시 다음 강의 계획도 있으실지요~? [sep] ...,NaN,BLANK,BLANK,BLANK
8,"어설픈 편법으로 (Airtable 샘플에 부족하긴 하지만), Midjourney-D...",NaN,BLANK,BLANK,BLANK
10,!! [sep] 작은 도움이라도 됐다면 제가 외려 감사하죠 [sep] ㅎㅎㅎ [se...,NaN,BLANK,BLANK,BLANK
11,! IFTTT는 조금 기능이 한정적인 느낌이라 쓰다가 말았었는데 Zapier도 한번...,NaN,BLANK,BLANK,BLANK
12,태현님 어제 세브방에 모집하신 스터디랑 다른 것인가요? [sep] 넵 지금 그 스터...,NaN,BLANK,BLANK,BLANK
16,정말 감사하게도 오늘 세미나를 해 주신 준용님께서 부방장을 맡아 주셨습니다 ~ 같이...,NaN,[https://www.youtube.com/watch?v=on623d5EfPw],Watch ChatGPT Build a Finance Startup,Let's build a demo Finance application using C...


In [ ]:
count = 0

#df to df_5, df_10, df_15, df_20
for index, row in df_5.iterrows():
    count += 1
    text = row['chat']

    inputs = ["summarize: " + text]

    inputs = tokenizer(inputs, max_length=max_input_length, truncation=True, return_tensors="pt").to('cuda')
    output = model.generate(**inputs, num_beams=8, do_sample=True, min_length=10, max_length=256)
    decoded_output = tokenizer.batch_decode(output, skip_special_tokens=True)[0]
    predicted_title = nltk.sent_tokenize(decoded_output.strip())[0]

    print(predicted_title)
    # 'summary'열 내용 채우기
    summary_data.loc[index, 'summary'] = predicted_title
    print(summary_data['summary'].count())
    if count > 100:
       break

# 새로운 DataFrame을 CSV 파일로 저장
# 시간 단위에 따라 바꿔주어야 함.
csv_path = '/content/drive/MyDrive/chat_test_5min(ver1.4).csv'
summary_data.to_csv(csv_path, index=False)

OpenAI와 microsoft가 미국에서 고소 당한 상태로 소송이 진행 중인데 캐나다에서는 현저한 노력을 해야 저작권을 인정해 주기 때문에 AI가 만들어낸 것들은 누구도 저작권이 인정되기 어려운 상태라고 한다.
1
웨비나에서 배우고 기억에 남은 것들 간략하게 정리해봤다.
2
강연 듣고 정리해야지 하면 안되더라구요.
3
Midjourney 쓰다보면 결과물 관리가 번거러운데 프로 모드에서 private 모드면 조금 낫겠지만 초보 입장에서 다양한 노하우가 보여서 너무 좋았다.
4
웨비나 평가 반영해서 업그레이드 해서 찾아 뵐 것이고 웨비나 평가 반영해서 웨비나 평가 반영해서 업그레이드 해서 찾아 뵐 것이다.
5
Discord의 midjourney 산출물 메시지 링크를 Discord 개인 서버에 쌓아두면 Zapier를 통해 노션 DB에 메시지 링크가 누적된다.
6
zapier는 IFTTT와 비슷한 기능이 있는 것인지 IFTTT 맛만 봤었지만 IFTTT와 비슷한 것 같다.
7
업무 자동화에 좋은 역할을 해줄 것 같은 느낌이 드는 Zapier와 같은 연동이라면 자피어가 사용감이 훨씬 좋다.
8
태현은 세브방에 모집한 스터디랑 다른 것인지 물었고 민트베어님에게 피드백을 받아야 할 것 같다고 했다.
9
ChatGPT로 사업하는데 관심 있어서 들어왔는데 세미나 다시 듣기도 가능한가요?
10
이방 갑자기 왜이렇게 커지고 있지 잘들어가집니다 잘들어가집니다 제이지님 잘들어가집니다 잘들어가집니다 제이지님 잘들어가집니다 제이지님 잘들어가집니다 제이지님 잘들어가집니다 제이지님 잘들어가집니다
11
Lexica Aperture로 만든 이미지를 상업적으로 사용하려면 팀 규모에 일부 제한이 있는 유료 플랜이 있어야 하며 5명 이상의 팀에는 Max 요금제가 필요하다.
12
chatgpt는 광범위하게 활용도가 넓어서 자료 리서치하는 입장에서도 어마어마해서 리서치에 활용할 때 인사이트를 줄 만한 답이 나오나 생각해볼만한 리서치 자료들을 막던져주는 비서같다고 해야하나
13
신재생 에너지 발전은 태양

### 중요 문장 추출

### 중요 단어 추출

In [ ]:
import locale
def getpreferredencoding(do_setlocale = True):
    return "UTF-8"
locale.getpreferredencoding = getpreferredencoding

!pip install sentence_transformers
!pip install konlpy

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
import numpy as np
import itertools

from konlpy.tag import Okt
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.metrics.pairwise import cosine_similarity
from sentence_transformers import SentenceTransformer

In [ ]:
text_data = pd.read_csv(csv_path)
text_data.head(5)

,text,summary,url,url_title,url_desc
0,맞습니다~ [sep] 저작권은 그럼 AI한테 있는걸까요? [sep] 아직 저작권은 ...,OpenAI와 microsoft가 미국에서 고소 당한 상태로 소송이 진행 중인데 캐...,BLANK,BLANK,BLANK
1,사진 [sep] 오늘 웨비나에서 배우고 기억에 남은 것들 간략하게 정리해봤어요.\n...,웨비나에서 배우고 기억에 남은 것들 간략하게 정리해봤다.,BLANK,BLANK,BLANK
2,"😄 [sep] 강연 듣고, 나중에 정리해야지- 하면 안되더라구요. 너무 멋집니다 [...",강연 듣고 정리해야지 하면 안되더라구요.,BLANK,BLANK,BLANK
3,! [sep] 이런것도 알려주셨나요? 이동하면서 들어서 중간중건에 못들어서요 [se...,Midjourney 쓰다보면 결과물 관리가 번거러운데 프로 모드에서 private ...,BLANK,BLANK,BLANK
4,오늘 참여 못해서 너무 아쉬운데 혹시 다음 강의 계획도 있으실지요~? [sep] ...,웨비나 평가 반영해서 업그레이드 해서 찾아 뵐 것이고 웨비나 평가 반영해서 웨비나 ...,BLANK,BLANK,BLANK


### notion database로 옮김

In [ ]:
!pip install python-dotenv # (※ 주의 : pip install dotenv가 아님)
!pip install notion-client

!dotenv set NOTION_TOKEN secret_Yp03AO7khgzix5l73o66WLWAIoQAJP8YPsu5y0RjWKV
!dotenv list

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
NOTION_TOKEN=secret_Yp03AO7khgzix5l73o66WLWAIoQAJP8YPsu5y0RjWKV
NOTION_TOKEN=secret_Yp03AO7khgzix5l73o66WLWAIoQAJP8YPsu5y0RjWKV


In [ ]:
from dotenv import dotenv_values
from notion_client import Client

config = dotenv_values(".env")
notion_secret = config.get('NOTION_TOKEN')
notion = Client(auth=notion_secret)

In [ ]:
import os
import requests
from pprint import pprint

notion_api_key = "secret_Yp03AO7khgzix5l73o66WLWAIoQAJP8YPsu5y0RjWKV"
# database 주소 바꿔야 함! ?v 앞의 주소
database_id = "91455b80ed624b3cbfcd23af027740f4"

headers = {
    "Authorization": f"Bearer {notion_api_key}",
    "Content-Type": "application/json",
    "Notion-Version": "2022-06-28"
}

url = f"https://api.notion.com/v1/pages"

In [ ]:
text_data = pd.read_csv(csv_path)
count = 0

okt = Okt()

for index, row in text_data.iterrows():
  count+=1
  doc = row['summary']
  text = row['text']
  text_url = row['url']
  url_title = row['url_title']
  url_desc = row['url_desc']

  if (len(doc)>len(text)):
      doc = text

  tokenized_doc = okt.pos(str(doc))

  nouns = [word[0] for word in tokenized_doc if (word[1] == 'Noun' or word[1] == 'Alpha') and len(word[1]) > 2]

  unique_words = list(set(nouns))
  word_list = []
  for line in unique_words:
      if len(line) > 2 and line != 'nan':
          word_list.append(line)
  print('명사추출 :', word_list)

  tag_list = [ ]
  for word in word_list:
      tag_list.append({"name": word})

  data = {
        "parent": {"database_id": database_id},
        "properties": {
            "Name": {
                "title": [
                    {
                        "text": {
                            "content": doc
                        }
                    }
                ]
            },
            # Add a multi-select property for tags
            "Tags": {
                "multi_select": tag_list
            },
            "Text": {
                "rich_text": [
                    {
                        "text": {
                            "content": text
                        }
                    }
                ]
            },
            "URL": {
                "rich_text": [
                    {
                        "text": {
                            "content": text_url
                        }
                    }
                ]
            },
            "Title": {
                "rich_text": [
                    {
                        "text": {
                            "content": url_title
                        }
                    }
                ]
            },
            "Desc": {
                "rich_text": [
                    {
                        "text": {
                            "content": url_desc
                        }
                    }
                ]
            },
            "Index": {
                "rich_text": [
                    {
                        "text": {
                            "content": str(count)
                        }
                    }
                ]
            },
        }
  }
  #print(data)
  response = requests.post(url, headers=headers, json=data)
  result = response.json()
  pprint(result)

  if count > 100:
    break

스트리밍 출력 내용이 길어서 마지막 5000줄이 삭제되었습니다.
                                                       'italic': False,
                                                       'strikethrough': False,
                                                       'underline': False},
                                       'href': None,
                                       'plain_text': "['https://youtu.be/TvwWxtkAdw8']",
                                       'text': {'content': "['https://youtu.be/TvwWxtkAdw8']",
                                                'link': None},
                                       'type': 'text'}],
                        'type': 'rich_text'}},
 'url': 'https://www.notion.so/sep-https-youtu-be-TvwWxtkAdw8-fde4afcecf4d4d1798ca018ac8ab8485'}
명사추출 : ['sep']
{'archived': False,
 'cover': None,
 'created_by': {'id': '65d3ebdf-83c0-48d9-be38-f4c9ec8688a1', 'object': 'user'},
 'created_time': '2023-05-31T10:47:00.000Z',
 'icon': None,
 'id': 'df549310-c017-4b98-bec8-0cc202806083

InvalidJSONError: ignored

1. database의 주소에서 ?v 앞이 database 주소
2. integration을 해주어야 함. -> add connection. 미리 등록한 integration 등록하기. : https://developers.notion.com/docs/create-a-notion-integration
